In [1]:
import requests, json, urllib.parse

In [2]:
base_url = 'https://itunes.apple.com/search'

artist = 'Mikhail Krug'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [3]:
# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [4]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

In [5]:
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [6]:
# Уникальные названия артистов
print(set(map(lambda x: x['artistName'], data)))

{'Mikhail Krug & Vika Tsyganova', 'Mikhail Krug & Gruppa Poputchik', 'Irina Krug & Mikhail Krug', 'Mikhail Krug', 'Mikhail Krug & Svetlana Ternova'}


In [7]:
# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

In [8]:
release_date_extractor = lambda x: x['releaseDate']

first_song = min(data, key=release_date_extractor)

print('First song:', first_song['trackName'], first_song['releaseDate'])

latest_song = max(data, key=release_date_extractor)

print('Last song:', latest_song['trackName'], latest_song['releaseDate'])

First song: Кольщик 1993-05-23 12:00:00+00:00
Last song: Приходите в мой дом 2020-08-12 12:00:00+00:00


In [9]:
# Функция переводящая миллисекунды в читаемый формат времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [10]:
time_extractor = lambda x: x['trackTimeMillis']
longest_song = max(data, key=time_extractor)

print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)

print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))

Longest song: Galley Gypsy Dance 0H5M51S
Shortest song: Late Rubl 0H2M12S


In [11]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

# Удаление переменной, действие опционально
del sorted_by_album

print(list(album_names.values()))

['Роза', 'Посвящение', 'Студентка', 'Duets', 'Мышка', 'Магадан', 'История любви', '50 Greatest Hits (Big Chanson Collection)', 'Любимой Женщине', 'Мой Бог', 'Golden Album - Greatest Hits', 'Legends of Russian Chanson. 20 Years', 'All Stars of Shanson', 'Golden Duets of Russian Chanson', 'Лучшие дуэты', 'Лучшие блатные дуэты', 'Десять лет спустя', '55 лет. Юбилейный альбом', 'Лучшие песни', 'Мадам', 'Жиганские песни', 'Исповедь', 'Владимирский централ', 'Лирика', 'Дискотека русского шансона']


In [12]:
album_songs_count_extractor = lambda x: len(x[1])

album_with_max_songs = max(album_groups.items(), key=album_songs_count_extractor)
print('Max album songs:', album_names[album_with_max_songs[0]], len(album_with_max_songs[1]))

album_with_min_songs = min(album_groups.items(), key=album_songs_count_extractor)
print('Min album songs:', album_names[album_with_min_songs[0]], len(album_with_min_songs[1]))

Max album songs: 50 Greatest Hits (Big Chanson Collection) 50
Min album songs: Мышка 1


In [13]:
album_durations = map(lambda x: (x[0], sum(map(time_extractor, x[1]))), album_groups.items())
album_durations = dict(album_durations)

album_with_max_duration = max(album_durations.items(), key=lambda x: x[1])
print('Max album duration:', album_names[album_with_max_duration[0]], convert_millis(album_with_max_duration[1]))

album_witn_min_duration = min(album_durations.items(), key=lambda x: x[1])
print('Min album duration:', album_names[album_witn_min_duration[0]], convert_millis(album_witn_min_duration[1]))

Max album duration: 50 Greatest Hits (Big Chanson Collection) 3H4M31S
Min album duration: Лучшие дуэты 0H3M0S


In [51]:
from statistics import mean, median
print(max([data[i]['artistName'] for i in range(len(data))], key=[data[i]['artistName'] for i in range(len(data))].count))
print(len(list(map(lambda x: x['trackId'],data))))
print(max(list(map(lambda x: x['primaryGenreName'], data)), key=list(map(lambda x: x['primaryGenreName'], data)).count))
print(len(set(map(lambda x: x['collectionId'], data))))
print(convert_millis(sum(list(map(lambda x: x['trackTimeMillis'], data)))))
print(convert_millis(round(mean(list(map(lambda x: x['trackTimeMillis'], data))))))
print(convert_millis(round(median(list(map(lambda x: x['trackTimeMillis'], data))))))
print(max(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], data)), key=lambda x: x[0])[1])
print(convert_millis(max(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], data)), key=lambda x: x[0])[0]))
print(min(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], data)), key=lambda x: x[0])[1])
print(convert_millis(min(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], data)), key=lambda x: x[0])[0]))

print((sum(list(map(lambda x: x['trackPrice'], data)))))
print(((mean(list(map(lambda x: x['trackPrice'], data))))))
print(((median(list(map(lambda x: x['trackPrice'], data))))))
print(max(list(map(lambda x: [x['trackPrice'], x['trackName']], data)), key=lambda x: x[0])[1])
print((max(list(map(lambda x: [x['trackPrice'], x['trackName']], data)), key=lambda x: x[0])[0]))
print(min(list(map(lambda x: [x['trackPrice'], x['trackName']], data)), key=lambda x: x[0])[1])
print((min(list(map(lambda x: [x['trackPrice'], x['trackName']], data)), key=lambda x: x[0])[0]))

Mikhail Krug
186
Chanson in Russian
25
12H1M6S
0H3M52S
0H3M48S
Galley Gypsy Dance
0H5M51S
Late Rubl
0H2M12S
238.43999999999966
1.2819354838709678
1.29
Come into My House (feat. Vika Tsyganova)
1.29
That Was Yesterday
0.99


In [53]:
from statistics import mean, median
album = [d for d in data if d['collectionName'] == 'Мадам']
print(album[0]['collectionName'])
print(len(list(map(lambda x: x['trackId'],album))))
print(max(list(map(lambda x: x['primaryGenreName'], album)), key=list(map(lambda x: x['primaryGenreName'], album)).count))
print(album[0]['releaseDate'])
print(convert_millis(sum(list(map(lambda x: x['trackTimeMillis'], album)))))
print(convert_millis(round(mean(list(map(lambda x: x['trackTimeMillis'], album))))))
print(convert_millis(round(median(list(map(lambda x: x['trackTimeMillis'], album))))))
print(max(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], album)), key=lambda x: x[0])[1])
print(convert_millis(max(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], album)), key=lambda x: x[0])[0]))
print(min(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], album)), key=lambda x: x[0])[1])
print(convert_millis(min(list(map(lambda x: [x['trackTimeMillis'], x['trackName']], album)), key=lambda x: x[0])[0]))

print((sum(list(map(lambda x: x['trackPrice'], album)))))
print(((mean(list(map(lambda x: x['trackPrice'], album))))))
print(((median(list(map(lambda x: x['trackPrice'], album))))))
print(max(list(map(lambda x: [x['trackPrice'], x['trackName']], album)), key=lambda x: x[0])[1])
print((max(list(map(lambda x: [x['trackPrice'], x['trackName']], album)), key=lambda x: x[0])[0]))
print(min(list(map(lambda x: [x['trackPrice'], x['trackName']], album)), key=lambda x: x[0])[1])
print((min(list(map(lambda x: [x['trackPrice'], x['trackName']], album)), key=lambda x: x[0])[0]))

Мадам
10
Chanson in Russian
1998-11-30 12:00:00+00:00
0H39M28S
0H3M56S
0H4M9S
Мой Бог (Маравихер-2)
0H5M35S
Здравствуйте
0H2M34S
12.899999999999999
1.29
1.29
Владимирский централ
1.29
Владимирский централ
1.29
